<a href="https://colab.research.google.com/github/OrionXV/WebtopAppForHateDetection/blob/main/Model/Notebook/HateSpeechModelCreator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
!pip install pytorch_lightning 
!pip install transformers
!pip install contractions
!pip install torchvision

     |████████████████████████████████| 750.6 MB 1.1 kB/s eta 0:00:018    |███▍                            | 79.7 MB 2.5 MB/s eta 0:04:24                | 179.0 MB 2.4 MB/s eta 0:03:5446      | 266.9 MB 263 kB/s eta 0:30:33     |████████████████▎               | 381.2 MB 114 kB/s eta 0:53:57     |████████████████████████        | 565.4 MB 2.7 MB/s eta 0:01:10     |██████████████████████████      | 609.6 MB 5.6 MB/s eta 0:00:26     |███████████████████████████████▉| 747.4 MB 3.6 MB/s eta 0:00:01
     |████████████████████████████████| 584 kB 309 kB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 409 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 136 kB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 661 kB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 4.9 MB 1.6 MB/s eta 0:00:01
     |██████████████████

     |████████████████████████████████| 4.0 MB 98 kB/s eta 0:00:017
     |████████████████████████████████| 880 kB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 6.6 MB 3.3 MB/s eta 0:00:01
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=de82344aa94d4f0ec0344400369186114c201b7935b85afe8e4169b42c394dea
  Stored in directory: /home/Standby-Coder/.cache/pip/wheels/12/1c/3d/46cf06718d63a32ff798a89594b61e7f345ab6b36d909ce033
Successfully built sacremoses


In [ ]:
#https://drive.google.com/file/d/14eEuuUnKI2wjrG7oYu2GpeaCdGz6PkCy/view?usp=sharing
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=14eEuuUnKI2wjrG7oYu2GpeaCdGz6PkCy' -O trial.zip 

--2022-05-07 11:07:36--  https://docs.google.com/uc?export=download&id=14eEuuUnKI2wjrG7oYu2GpeaCdGz6PkCy
Resolving docs.google.com (docs.google.com)... 142.250.77.142, 2404:6800:4007:822::200e
Connecting to docs.google.com (docs.google.com)|142.250.77.142|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-a0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/l7rtv9b9erkfaocdg7c37jcg05oe55oq/1651901850000/04989237414475792611/*/14eEuuUnKI2wjrG7oYu2GpeaCdGz6PkCy?e=download [following]
--2022-05-07 11:07:41--  https://doc-10-a0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/l7rtv9b9erkfaocdg7c37jcg05oe55oq/1651901850000/04989237414475792611/*/14eEuuUnKI2wjrG7oYu2GpeaCdGz6PkCy?e=download
Resolving doc-10-a0-docs.googleusercontent.com (doc-10-a0-docs.googleusercontent.com)... 142.250.182.97, 2404:6800:4009:80c::2001
Connecting to doc-10-a0-docs.googleusercontent.com (doc-10-a0-docs.googleusercon

In [ ]:
!unzip -q trial.zip

In [ ]:
import numpy as np
import pandas as pd
import random
import seaborn as sns  
sns.set()
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')

%config InLineBackend.figure_format = 'retina'
%matplotlib inline

import csv
import json
import logging

from pathlib import Path
import os
import glob

import tarfile
import tempfile

from tqdm import tqdm
import torch                    
import torchvision
import pytorch_lightning as pl
from torchmetrics import functional as FM 
from sklearn.model_selection import train_test_split

In [ ]:
path = '/content'
#path = '/mnt/f/Git repos/HateDetector/Model/Notebook'
extension = 'csv'
os.chdir(path)
result = glob.glob('*.{}'.format(extension))
print(result)

['labeled_data.csv']


In [ ]:
data_path = path + '/' + result[0]

In [ ]:
data = pd.read_csv(data_path, usecols = ['class', 'tweet'])
data.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [ ]:
data.rename(columns={'tweet' : 'text',
                   'class' : 'label'}, 
                    inplace=True)

In [ ]:
#0 = hate speech, 1 = offensive lang, 2 = neither
data.groupby('label').count()

,text
label,
0,1430
1,19190
2,4163


In [ ]:
data.label

0        2
1        1
2        1
3        1
4        1
        ..
24778    1
24779    2
24780    1
24781    1
24782    2
Name: label, Length: 24783, dtype: int64

In [ ]:
import re
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import spacy
import contractions

def textCleaner(mystring):
    mystring = " ".join(mystring.split())
    nystring = contractions.fix(mystring)
    #re.sub(r'(!|.)1+', '', text1) 
    return re.sub(r"^\W+", "", mystring)

def textCleaner_adv(mystring):
    newstring = " ".join(x.lower() for x in mystring.split())
    
    newstring = newstring.replace('[^\w\s]','')
    
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    newstring = emoji_pattern.sub(r'', newstring)
    
    stop = stopwords.words('english')
    newstring = " ".join(x for x in newstring.split() if x not in stop)
    
    nlp = spacy.load('en', disable=['parser', 'ner'])
    doc = nlp(newstring)

    return " ".join([token.lemma_ for token in doc])

In [ ]:
data['text'].apply(lambda x: textCleaner(x))

0        RT @mayasolovely: As a woman you shouldn't com...
1        RT @mleew17: boy dats cold...tyga dwn bad for ...
2        RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: Y...
3        RT @C_G_Anderson: @viva_based she look like a ...
4        RT @ShenikaRoberts: The shit you hear about me...
                               ...                        
24778    you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779    you've gone and broke the wrong heart baby, an...
24780    young buck wanna eat!!.. dat nigguh like I ain...
24781                youu got wild bitches tellin you lies
24782    Ruffled | Ntac Eileen Dahlia - Beautiful color...
Name: text, Length: 24783, dtype: object

In [ ]:
X_train_, X_test, y_train_, y_test = train_test_split(
    data.index.values,
    data.label.values,
    test_size=0.10,
    random_state=42,
    stratify=data.label.values,    
)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    data.loc[X_train_].index.values,
    data.loc[X_train_].label.values,
    test_size=0.10,
    random_state=42,
    stratify=data.loc[X_train_].label.values,  
)

In [ ]:
data['data_type'] = ['not_set']*data.shape[0]
data.loc[X_train, 'data_type'] = 'train'
data.loc[X_val, 'data_type'] = 'val'
data.loc[X_test, 'data_type'] = 'test'

In [ ]:
data.groupby(['label', 'data_type']).count()

text
label data_type       
0     test         143
      train       1158
      val          129
1     test        1920
      train      15543
      val         1727
2     test         416
      train       3372
      val          375

In [ ]:
train = data.loc[data["data_type"]=="train"]
train.head()

,label,text,data_type
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,train
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,train
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,train
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,train
6,1,"!!!!!!""@__BrighterDays: I can not just sit up ...",train


In [ ]:
test = data.loc[data["data_type"]=="test"]
test.head()

,label,text,data_type
0,2,!!! RT @mayasolovely: As a woman you shouldn't...,test
12,1,""" So hoes that smoke are losers ? "" yea ... go...",test
14,1,""" bitch get up off me """,test
17,1,""" bitch who do you love """,test
25,1,""" her pussy lips like Heaven doors "" &#128524;",test


In [ ]:
val = data.loc[data["data_type"]=="val"]
val.head()

,label,text,data_type
5,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just...",val
27,1,""" i met that pussy on Ocean Dr . i gave that p...",val
31,1,""" i'd say im back to the old me but my old bit...",val
44,1,""" post a picture of that pussy get 200 likes """,val
46,1,""" quick piece of pussy call it a drive by """,val


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

example_text = train.loc[3, 'text']
bert_input = tokenizer(
    example_text,padding='max_length', 
    max_length = 512, 
    truncation=True, 
    return_tensors="pt")


print(bert_input['input_ids'])
print(bert_input['token_type_ids'])
print(bert_input['attention_mask'])

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

tensor([[  101,   106,   106,   106,   106,   106,   106,   106,   106,   106,
           155,  1942,   137,   140,   168,   144,   168,  4347,   131,   137,
           191, 12416,   168,  1359,  1131,  1440,  1176,   170,   189,  4047,
          3382,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [ ]:
example_text = tokenizer.decode(bert_input.input_ids[0])

print(example_text)

[CLS]!!!!!!!!! RT @ C _ G _ Anderson : @ viva _ based she look like a tranny [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [ ]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):
        self.labels = list(df['label'])
        self.texts =[tokenizer(
            text, 
            padding='max_length', 
            max_length = 256, 
            truncation=True,
            return_tensors="pt") 
        for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [ ]:
'''weight_for_0 = (1 / hate)*(total)/3.0 
weight_for_1 = (1 / ofensive)*(total)/3.0
weight_for_2 = (1 / neither)*(total)/3.0


class_weight = {0: weight_for_0, 1: weight_for_1, 2: weight_for_2}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))
print('Weight for class 2: {:.2f}'.format(weight_for_2))'''

"weight_for_0 = (1 / hate)*(total)/3.0 \nweight_for_1 = (1 / ofensive)*(total)/3.0\nweight_for_2 = (1 / neither)*(total)/3.0\n\n\nclass_weight = {0: weight_for_0, 1: weight_for_1, 2: weight_for_2}\n\nprint('Weight for class 0: {:.2f}'.format(weight_for_0))\nprint('Weight for class 1: {:.2f}'.format(weight_for_1))\nprint('Weight for class 2: {:.2f}'.format(weight_for_2))"

In [ ]:
initial_output_bias = np.array([3.938462, 15, 5.])
initial_output_bias 

array([ 3.938462, 15.      ,  5.      ])

In [ ]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.1):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 3, bias = True)
        #self.linear.apply(
        self.relu = nn.ReLU()
        #initial_output_bias = np.array([[3.938462]*768, [15]*768, [5.]*768])
        initial_output_bias = np.array([3.938462, 15, 5.])
        K = torch.Tensor(initial_output_bias)
        #K = torch.unsqueeze(torch.unsqueeze(torch.unsqueeze(K,0),0),0)
        #with torch.no_grad():
        #    with torch.no_grad():
        #        self.linear.weight = torch.nn.Parameter(K)
        print(self.linear.bias)
        print(self.linear.bias.data)
        with torch.no_grad():
            self.linear.bias.data = self.linear.bias.data + K
        print(self.linear.bias)
        print(self.linear.bias.data)

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids = input_id, attention_mask = mask, return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [ ]:

#train_set, val_set = Dataset(train), Dataset(val)
#train_dataloader = DataLoader(train_set, batch_size=5, shuffle=True)
#val_dataloader = DataLoader(val_set, batch_size=5)

In [ ]:
'''
class TextModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        #self.train = train
        #self.val = val
        self.model = self._build_model()
        
    def forward(self, input_id, mask):
        # in lightning, forward defines the prediction/inference actions 
        return self.model(input_id, mask)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def training_step(self, batch, batch_idx):
        # training_step defined the train loop.
        # It is independent of forward
        x, y = batch
        output = self.forward(
            x['input_ids'], 
            x['attention_mask']
            )
        loss = nn.CrossEntropyLoss(output, y)
        # Logging to TensorBoard by default
        self.log("train_loss", loss, on_epoch = True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        preds = self.eval().forward(
            x['input_ids'], 
            x['attention_mask']
        )

        loss = nn.CrossEntropyLoss(preds, y)
        self.log("val_loss", loss)
        
        #return loss
    
    def train_dataloader(self, train):
        return train
    
    def val_dataloader(self, val):
        return val

    def _build_model(self):
        return BertClassifier(
            dropout = 0.1
        )
'''

'\nclass TextModel(pl.LightningModule):\n    def __init__(self):\n        super().__init__()\n        #self.train = train\n        #self.val = val\n        self.model = self._build_model()\n        \n    def forward(self, input_id, mask):\n        # in lightning, forward defines the prediction/inference actions \n        return self.model(input_id, mask)\n\n    def configure_optimizers(self):\n        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)\n        return optimizer\n\n    def training_step(self, batch, batch_idx):\n        # training_step defined the train loop.\n        # It is independent of forward\n        x, y = batch\n        output = self.forward(\n            x[\'input_ids\'], \n            x[\'attention_mask\']\n            )\n        loss = nn.CrossEntropyLoss(output, y)\n        # Logging to TensorBoard by default\n        self.log("train_loss", loss, on_epoch = True)\n        return loss\n    \n    def validation_step(self, batch, batch_idx):\n        x, y

In [ ]:
from torch.optim import Adam
from tqdm import tqdm
from torch.utils.data import DataLoader

def trainFunc(model, train_data, val_data, learning_rate, epochs):

    train = Dataset(train_data.copy()) 
    val = Dataset(val_data.copy())

    train_dataloader = DataLoader(train, batch_size=8, shuffle=True)
    val_dataloader = DataLoader(val, batch_size=8)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    print(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    for epoch_num in range(epochs):

        total_acc_train = 0
        total_loss_train = 0

        for train_input, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)
            
            batch_loss = criterion(output, train_label)
            total_loss_train += batch_loss.item()
            
            acc = (output.argmax(dim=1) == train_label).sum().item()
            total_acc_train += acc

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()
        
        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():

            for val_input, val_label in val_dataloader:

                val_label = val_label.to(device)
                mask = val_input['attention_mask'].to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)

                batch_loss = criterion(output, val_label)
                total_loss_val += batch_loss.item()
                
                acc = (output.argmax(dim=1) == val_label).sum().item()
                total_acc_val += acc
        
        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
            | Train Accuracy: {total_acc_train / len(train_data): .3f} \
            | Val Loss: {total_loss_val / len(val_data): .3f} \
            | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                


In [ ]:
EPOCHS = 5
model = BertClassifier()
LR = 1e-5
              
trainFunc(model = model, train_data = train, val_data = val, learning_rate = LR, epochs = EPOCHS)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Parameter containing:
tensor([-0.0343,  0.0013, -0.0349], requires_grad=True)
tensor([-0.0343,  0.0013, -0.0349])
Parameter containing:
tensor([ 3.9041, 15.0013,  4.9651], requires_grad=True)
tensor([ 3.9041, 15.0013,  4.9651])


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2510/2510 [27:38<00:00,  1.51it/s]


Epochs: 1 | Train Loss:  0.044             | Train Accuracy:  0.892             | Val Loss:  0.029             | Val Accuracy:  0.917


...

Epochs: 3 | Train Loss:  0.016             | Train Accuracy:  0.952             | Val Loss:  0.031             | Val Accuracy:  0.919


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2510/2510 [27:40<00:00,  1.51it/s]


Epochs: 4 | Train Loss:  0.006             | Train Accuracy:  0.985             | Val Loss:  0.043             | Val Accuracy:  0.917


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2510/2510 [26:56<00:00,  1.55it/s]


Epochs: 5 | Train Loss:  0.003             | Train Accuracy:  0.993             | Val Loss:  0.051             | Val Accuracy:  0.918


In [ ]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=8)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    print(device)
    
    if use_cuda:
        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

            test_label = test_label.to(device)
            mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)

            acc = (output.argmax(dim=1) == test_label).sum().item()
            total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [ ]:
evaluate(model, test)

cuda
Test Accuracy:  0.910


In [ ]:
nb_classes = 3
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    val_set = Dataset(val.copy())
    val_dataloader = DataLoader(val_set, batch_size=8)
    for i, batch in enumerate(val_dataloader):
        #inputs = inputs.to(device)
        input, classes = batch
        mask = input['attention_mask'].to(device)
        input_id = input['input_ids'].squeeze(1).to(device)
        classes = classes.to(device)
        outputs = model(input_id, mask)
        preds = outputs.argmax(dim=1)
        for t, p in zip(classes, preds):
            confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

cuda
tensor([[  67.,   53.,    9.],
        [  57., 1632.,   38.],
        [  10.,   15.,  350.]])


In [ ]:
outputs

tensor([[10.0182, 11.0854,  2.4651],
        [ 6.0774, 13.4465,  4.3787],
        [ 5.0863, 14.4312,  4.1863],
        [11.7804,  7.6370,  3.9360],
        [ 7.4148,  5.7695, 10.7023],
        [ 6.7285, 13.3682,  3.5280],
        [ 3.8250, 15.8185,  4.0142]], device='cuda:0')

In [ ]:
nb_classes = 3
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    test_set = Dataset(test)
    test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=8)
    for i, (inputs, classes) in enumerate(test_dataloader):
        mask = input['attention_mask'].to(device)
        input_id = input['input_ids'].squeeze(1).to(device)
        classes = classes.to(device)
        outputs = model(input_id, mask)
        preds = outputs.argmax(dim=1)
        for t, p in zip(classes, preds):
            confusion_matrix[t.long(), p.long()] += 1


print(confusion_matrix)

tensor([[  18.,   73.,   20.],
        [ 240., 1224.,  229.],
        [  52.,  253.,   61.]])


In [ ]:
torch.save(model, path+'/TextModel.pt')

In [ ]:
from google.colab import files
files.download('/content/TextModel.pt') 

ModuleNotFoundError: No module named 'google.colab'